In [99]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [100]:
data=pd.read_csv("./NationalNames.csv")

In [101]:
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [102]:
#take only the unique ones,suppose one name can be both used by male female, in that case we will take the value of gender 1 and , depending on how many were given to male or female 
label=LabelEncoder()
data["Gender"]=label.fit_transform(data.Gender)

In [103]:
df=data.groupby(["Name"]).mean()[["Gender"]].reset_index()
df

,Name,Gender
0,Aaban,1.0
1,Aabha,0.0
2,Aabid,1.0
3,Aabriella,0.0
4,Aadam,1.0
...,...,...
93884,Zytavious,1.0
93885,Zyvion,1.0
93886,Zyyanna,0.0
93887,Zyyon,1.0


In [104]:
import string

In [105]:
letters=string.ascii_lowercase #all lower case alphabets

In [106]:
vocab=dict(zip(letters,range(1,27)))

In [107]:
vocab

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [108]:
rev_vocab=dict(zip(range(1,27),letters))

In [109]:
rev_vocab

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

In [110]:
name="Elizabeth"
[vocab[ch] for ch in name.lower()]

[5, 12, 9, 26, 1, 2, 5, 20, 8]

In [111]:
seq=[vocab[ch] for ch in name.lower()]
[rev_vocab[ch] for ch in seq]

['e', 'l', 'i', 'z', 'a', 'b', 'e', 't', 'h']

In [112]:
X=[]
for name in df.Name:
    X.append([vocab[ch] for ch in name.lower()])
    

In [113]:
y=df.Gender

In [114]:
#hum naam ko as a sequence of numbers se analyze krenge

In [115]:
#X is a list of list. All the names should be of same length
#for processing in rnn, so we will use padding before inputting in rnn

In [116]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [117]:
X=pad_sequences(X,maxlen=10) #to make all lengths equal by padding of zeros

In [118]:
X.shape,y.shape #consists of all unique names

((93889, 10), (93889,))

In [119]:
from tensorflow.keras.layers import Input, Dense,SimpleRNN,Embedding
from tensorflow.keras.models import Model
#word embedding concept

In [120]:
in_layer=Input(shape=(10,))
embedding=Embedding(input_dim=27,output_dim=5)(in_layer) #characters-->27 #5 dimensional space main ocnvert krdiya
#10 rows 5 columns
rnn=SimpleRNN(units=30)(embedding)
out_layer=Dense(units=1,activation="sigmoid")(rnn)
model=Model(in_layer,out_layer)

In [121]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 5)             135       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 30)                1080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 1,246
Trainable params: 1,246
Non-trainable params: 0
_________________________________________________________________


In [125]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [136]:
model.fit(X,y.values,batch_size=1000,epochs=60)

Epoch 1/60
94/94 [==============================] - 1s 8ms/step - loss: 0.3633 - accuracy: 0.7721
Epoch 2/60
94/94 [==============================] - 1s 8ms/step - loss: 0.3626 - accuracy: 0.7726
Epoch 3/60
94/94 [==============================] - 1s 8ms/step - loss: 0.3622 - accuracy: 0.7725
Epoch 4/60
94/94 [==============================] - 1s 8ms/step - loss: 0.3613 - accuracy: 0.7735
Epoch 5/60
94/94 [==============================] - 1s 7ms/step - loss: 0.3610 - accuracy: 0.7735
Epoch 6/60
94/94 [==============================] - 1s 7ms/step - loss: 0.3604 - accuracy: 0.7740
Epoch 7/60
94/94 [==============================] - 1s 8ms/step - loss: 0.3594 - accuracy: 0.7746
Epoch 8/60
94/94 [==============================] - 1s 7ms/step - loss: 0.3591 - accuracy: 0.7744
Epoch 9/60
94/94 [==============================] - 1s 7ms/step - loss: 0.3585 - accuracy: 0.7750
Epoch 10/60
94/94 [==============================] - 1s 8ms/step - loss: 0.3577 - accuracy: 0.7754
Epoch 11/60
94/94 [

In [153]:
name="Seeta"
seq=[vocab[ch] for ch in name.lower()]
x_test=pad_sequences([seq],maxlen=10)
predict=model.predict(x_test)
if predict[0]>0.5:
    print("Male")
else: print("Female")

Female
